In [22]:
import numpy as np

In [32]:
import pandas as pd

In [46]:
import xarray as xr

In [58]:
import urllib

In [62]:
# Choose iceberg year (2002 - 2015 available)
# Note: Iceberg Season starts in November so many datasets include dates from year-1
season_year = 2015
iip_url_base = 'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/' 
iip_filename = 'IIP_{}IcebergSeason.csv'.format(season_year)
iip_url = iip_url_base + iip_filename
r = urllib.request.urlretrieve(iip_url)
iip_df = pd.read_csv(r[0], converters={'TIME':str})
iip_df['DATETIME'] = pd.to_datetime(df['DATE'] + 'T' + df['TIME'])

In [63]:
iip_df.head()

,ICE-YEAR,BERG_NUMBER,DATE,TIME,LATITUDE,LONGITUDE,METHOD,SIZE,SHAPE,SOURCE,DATETIME
0,2015,1,11/21/2014,1639,59.98,-63.44,VIS,GEN,GEN,GCFR,2014-11-21 16:39:00
1,2015,1,12/13/2014,2138,58.32,-61.18,SAT-HIGH,RAD,RAD,RSA2,2014-12-13 21:38:00
2,2015,2,11/29/2014,2147,58.57,-60.53,SAT-HIGH,RAD,RAD,RSA2,2014-11-29 21:47:00
3,2015,2,12/6/2014,2142,58.21,-60.05,SAT-HIGH,RAD,RAD,RSA2,2014-12-06 21:42:00
4,2015,2,12/16/2014,1553,57.69,-59.16,VIS,LG,GEN,GTJZ,2014-12-16 15:53:00


In [66]:
# Choose the min number of observations for an eligible iceberg
min_num_obs = 10
eligible_bergs = np.asarray(
    iip_df['BERG_NUMBER'].value_counts()\
    .loc[iip_df['BERG_NUMBER'].value_counts() > min_num_obs].index)

In [69]:
chosen_inds_arr = []

for i in range(eligible_bergs.size):

    iip_berg_id = eligible_bergs[i]
    iip_berg_df = iip_df.loc[iip_df['BERG_NUMBER'] == iip_berg_id]
    
    ind0 = iip_berg_df.index.tolist()[0]
    indf = iip_berg_df.index.tolist()[-1]
    
    max_time_dif = np.timedelta64(24*60*3, 'm')
    
    chosen_inds = []

    for j in range(len(iip_berg_df)-1):

        time_dif = (iip_berg_df.DATETIME.values[j+1] - \
                    iip_berg_df.DATETIME.values[j]).astype('timedelta64[m]')
        
        if time_dif < max_time_dif:
            chosen_inds.append(j+ind0)

        elif len(chosen_inds) > 5:
            chosen_inds_arr.append(chosen_inds)
            chosen_inds = []
        else:
            chosen_inds = []

    if len(chosen_inds) > 5:
        chosen_inds_arr.append(chosen_inds)

In [74]:
# Choose which iceberg track to look at (index)
chosen_track_ind = 2

iip_berg_df = df.loc[chosen_inds_arr[chosen_track_ind]].reset_index()

In [75]:
iip_berg_df.head()

,index,ICE-YEAR,BERG_NUMBER,DATE,TIME,LATITUDE,LONGITUDE,METHOD,SIZE,SHAPE,SOURCE,DATETIME
0,4986,2015,21165,6/23/2015,1227,47.67,-48.80,R/V,GEN,GEN,GTJZ,2015-06-23 12:27:00
1,4987,2015,21165,6/23/2015,1914,47.59,-48.81,R/V,GEN,GEN,GTJZ,2015-06-23 19:14:00
2,4988,2015,21165,6/24/2015,1452,47.52,-48.84,R/V,SM,GEN,GTJZ,2015-06-24 14:52:00
3,4989,2015,21165,6/26/2015,1643,47.34,-48.72,R/V,SM,GEN,GMRS,2015-06-26 16:43:00
4,4990,2015,21165,6/27/2015,1116,47.32,-48.72,R/V,SM,GEN,GPGR,2015-06-27 11:16:00


In [78]:
class Iceberg:
    
    def __init__(self, id_num, times, lats, lons, size):
        self.id_num = id_num
        self.times = times
        self.times_ref0 = self.get_times_ref0(self.times)
        self.lats = lats
        self.lons = lons
        
        if type(size) == str:
            self.length, self.width, self.height = self.get_berg_dims(size)
        elif type(size) == list and len(size) == 3:
            self.length, self.width, self.height = size[0], size[1], size[2]
        else:
            print('Invalid size argument')
            
    def get_times_ref0(self, times):
        times_ref0 = []
        t_offset = times[0].hour + times[0].minute/60  # from start of day
        for i in range(len(times)):
            times_ref0.append(round((times[i] - times[0]).days*24 + \
                                    (times[i] - times[0]).seconds/3600 + t_offset, 1))
        return times_ref0
    
    def set_times_ref0(self, times):
        self.times_ref0 = self.get_times_ref0(times)
            
    def get_berg_dims(self, size):
        # Size must be GR, BB, SM, MED, LG, VLG
        # See https://nsidc.org/data/g00807 for more info
        if size == 'GR':
            l = (0+5)/2; w = (0+5)/2; h = (0+1)/2*10
        elif size == 'BB':
            l = (5+15)/2; w = (5+15)/2; h = (1+5)/2*10        
        elif size == 'SM':
            l = (15+60)/2; w = (15+60)/2; h = (5+15)/2*10        
        elif size == 'MED':
            l = (60+120)/2; w = (60+120)/2; h = (15+45)/2*10               
        elif size == 'LG':
            l = (120)/2; w = (120)/2; h = (45+75)/2*10                
        elif size == 'VLG':
            # Sizes have no listed upper bound
            l = (200+200/2)/2; w = (200+200/2)/2; h = (75+75/2)/2*10     
        # This info for GEN is wrong!
        elif size == 'GEN':
            l = (120)/2; w = (120)/2; h = (45+75)/2*10            
        else:
            print('unknown size class')
            l = None; w = None; h = None
        return l, w, h

In [79]:
iip_berg = Iceberg(iip_berg_df['BERG_NUMBER'].loc[0],
                   iip_berg_df['DATETIME'].loc[0:1].tolist(),
                   iip_berg_df['LATITUDE'].loc[0:1].tolist(),
                   iip_berg_df['LONGITUDE'].loc[0:1].tolist(),
                   iip_berg_df['SIZE'].loc[0])

In [84]:
min(iip_berg.lats)

47.59

In [105]:
class OceanModel(object):
    def __init__(self, x_min, x_max, y_min, y_max, t_min, t_max):
        self.x_min = x_min - abs(x_min-x_max) - self.xy_res
        self.x_max = x_max + abs(x_min-x_max) + self.xy_res
        self.y_min = y_min - abs(y_min-y_max) - self.xy_res
        self.y_max = y_max + abs(y_min-y_max) + self.xy_res
        if self.t_units == 'hours since 2000-01-01 00:00:00':
            self.t_min = (t_min - pd.Timestamp('2000-01-01')).days*24 + \
                         (t_min - pd.Timestamp('2000-01-01')).seconds/3600 - self.t_res
            self.t_max = (t_max - pd.Timestamp('2000-01-01')).days*24 + \
                         (t_max - pd.Timestamp('2000-01-01')).seconds/3600 + self.t_res               

In [106]:
class GLBv008(OceanModel):
    url = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_56.3'
    xy_res = 0.08  # spatial resolution in degrees lat/lon
    t_res = 3  # temporal resolution in hours
    t_units = 'hours since 2000-01-01 00:00:00'
    def __init__(self, x_min, x_max, y_min, y_max, t_min, t_max):
        super().__init__(x_min, x_max, y_min, y_max, t_min, t_max)
        self.ds = xr.open_dataset(self.url, decode_times=False).sel(depth=0.0, 
                  lat = slice(self.y_min, self.y_max), 
                  lon = slice(self.x_min, self.x_max), 
                  time = slice(self.t_min, self.t_max))

In [107]:
testobj = GLBv008(min(iip_berg.lons), max(iip_berg.lons),
                  min(iip_berg.lats), max(iip_berg.lats),
                  iip_berg.times[0], iip_berg.times[-1])

In [108]:
testobj.ds

<xarray.Dataset>
Dimensions:            (lat: 10, lon: 3, time: 4)
Coordinates:
    depth              float64 0.0
  * lat                (lat) float64 47.44 47.48 47.52 47.56 47.6 47.64 ...
  * lon                (lon) float64 -48.88 -48.8 -48.72
  * time               (time) float64 1.357e+05 1.357e+05 1.357e+05 1.357e+05
Data variables:
    tau                (time) float64 ...
    water_u            (time, lat, lon) float64 ...
    water_u_bottom     (time, lat, lon) float64 ...
    water_v            (time, lat, lon) float64 ...
    water_v_bottom     (time, lat, lon) float64 ...
    water_temp         (time, lat, lon) float64 ...
    water_temp_bottom  (time, lat, lon) float64 ...
    salinity           (time, lat, lon) float64 ...
    salinity_bottom    (time, lat, lon) float64 ...
    surf_el            (time, lat, lon) float64 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgra